In [115]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [116]:
%matplotlib notebook

#<GRADED>
import numpy as np
from numpy.matlib import repmat
import sys
import time
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets

from scipy.stats import linregress

# new torch imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# # new vision-dataset-related torch imports
# import torchvision
# import torchvision.datasets as dset
# import torchvision.transforms as transforms

# misc imports
import random
#</GRADED>

Brainstorm:

1. Before Training: Use PCA to reduce the number of features (Low priority)

    1.1 Things such as the dog ID should not make a difference

2. Training: 

    2.1 Use deep neutral network to train a model and make predictions
    
    2.2 Use boosting and bagging to improve the model (Low priority)

3. Things to consider:

    3.1 Need to choose loss functions
    
    3.2 We should numericalize and normalize all features. Need to come up with ways to represent things such as "date of birth" and "N/A" data entries as numbers.
    


# STEP 1: Data Loading and Cleaning

In [117]:
# Choose from LF, LH, RF, RH 
leg = "RH"

x_file_name = leg + "_train.csv"

x_lf = pd.read_csv(x_file_name)

y_lf = x_lf[leg]



In [118]:
def clean_up_my_data_bro(x_lf, leg = "POTATO", data_type = "test"):
    
    if data_type=="test":
        x_lf = x_lf.drop(labels=["id","dob","forceplate_date"], axis=1) # these should not matter
    elif data_type=="train":
        x_lf = x_lf.drop(labels=["id","dob","forceplate_date",leg], axis=1) # these should not matter
    else:
        print("wrong data_type bro")
        
    
    # if gait is walk then 1 else -1
    for i in range(0,len(x_lf['gait'])):
        if x_lf['gait'][i] == "Walk":
            x_lf['gait'][i] = 1
        else:
            x_lf['gait'][i]= -1
    
    
    for i in range(0,len(x_lf['speed'])):
        if x_lf['speed'][i] == "Not able to walk":
            x_lf['speed'][i] = 0
        if x_lf['speed'][i] == "Not able to trot":
            x_lf['speed'][i] = -1

    x_lf = np.array(x_lf)
            
    col_remove = []
    print("x_training col number ", len(x_lf[0]))
    
    
    for j in range(0,len(x_lf[0])):

        col_sum =0
        col_count =0

        for i in range(0,len(x_lf)):
            
            if j==185:
                if x_lf[i][j]=="Trot":
                    x_lf[i][j]=1
            
            if x_lf[i][j]=="Not able to trot":
                x_lf[i][j]=float("nan")
        
            if x_lf[i][j]=="no valid trials":
                x_lf[i][j]= float("nan")
        
            if x_lf[i][j] == "No Data" or x_lf[i][j]=="no data":
                x_lf[i][j] = float("nan")


            if isinstance(x_lf[i][j], str)==True:
                #print("String, " + str(x_lf[i][j]) +", is converted to float")
                x_lf[i][j] = float(x_lf[i][j])    


            
            if np.isnan(x_lf[i][j])==False:
                col_sum += x_lf[i][j]
                col_count += 1


        col_mean = col_sum/col_count
        col_var = 0
        
        

        for i in range(0,len(x_lf)):
            if np.isnan(x_lf[i][j]):
                #print("Nan, " + str(x_lf[i][j]) +", is set to the column mean")
                x_lf[i][j]=col_mean
                
            else:  
                col_var += (x_lf[i][j]-col_mean)**2
            

        col_std = (col_var/float(col_count))**(1/2)

        if col_std == 0:
            col_remove.append(j) 
            col_std = 1
            print("removed column ", j, " bc all data are the same")

        x_lf[:,j] -= col_mean
        x_lf[:,j] /= col_std
    
    

    for i in col_remove:
        x_lf = np.delete(x_lf,obj=i,axis=1)

    print("x_lf and y_lf now has " + str(len(x_lf[0])) + " columns")

    x_lf2 = x_lf # Cleaned_Data
    
    # CLIPPING
    
    x_lf2 = np.clip(x_lf2,-2,2)
    
    return x_lf2


In [119]:
# x_lf = x_lf.drop(labels=["id","dob","forceplate_date","RH"], axis=1) # these should not matter


In [120]:
# for i in x_lf.columns:
#     print(i)


In [121]:
x_lf2 = clean_up_my_data_bro(x_lf, leg=leg, data_type = "train")

x_training col number  367
removed column  3  bc all data are the same
removed column  185  bc all data are the same
x_lf and y_lf now has 365 columns


<ipython-input-118-da01bc06e466>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['gait'][i] = 1


In [122]:
# for i in x_lf2:
#     for j in i:
#         print(j)

In [123]:
# columns_to_plot = 70

# x = np.arange(0,len(x_lf2))
# y = x_lf2[:,columns_to_plot]

# plt.scatter(x,y)
# plt.show()

In [124]:
# let labels be -1 and 1


# for i in range(0,len(y_lf)):
#     if y_lf[i]==0:
#         y_lf[i]=-1

y_lf2 = np.array(y_lf)

# STEP 2: Train a Model

# 2.1: Using P8 Code

In [125]:

class MLPNet(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim=1):
        super(MLPNet, self).__init__()
        """ pytorch optimizer checks for the properties of the model, and if
            the torch.nn.Parameter requires gradient, then the model will update
            the parameters automatically.
        """
        
        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim

        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)


In [126]:
def softmax_probs(probs):
    probs = torch.exp(probs)
    sum_probs = torch.sum(probs, axis=-1, keepdims=True)
    probs = probs/sum_probs
    return probs

In [127]:
# # Training function
# #<GRADED>
# def train_classification_model( x, y, model, num_epochs, lr=1e-1, print_freq=100):
#     """Train loop for a neural network model. Please use the SGD optimizer, optim.SGD.
    
#     Input:
#         train_loader:    Data loader for the train set. 
#                          Enumerate through to train with each batch.
#         model:           nn.Model to be trained
#         num_epochs:      number of epochs to train the model for
#         lr:              learning rate for the optimizer
#         print_freq:      frequency to display the loss
    
#     Output:
#         model:   nn.Module trained model
#     """
#     optimizer = optim.SGD(model.parameters(), lr=lr)  # create an SGD optimizer for the model parameters
#     for epoch in range(num_epochs):
#         loss = 0

#         # Iterate through the dataloader for each epoch
#         #for batch_idx, (imgs, labels) in enumerate(train_loader):
#         for i in range(len(x)):
#             # imgs (torch.Tensor):    batch of input images
#             # labels (torch.Tensor):  batch labels corresponding to the inputs
            
#             # Implement the training loop using imgs, labels, and cross entropy loss
            
#             imgs = torch.tensor(x[i].astype(np.float32))
#             labels = torch.tensor(y[i].astype(np.float32)).long()
            
            
            
#             optimizer.zero_grad()
#             preds = model(imgs)
#             preds = softmax_probs(preds)
#             loss += nn.functional.cross_entropy(preds, labels)
#             #loss = mse_loss(preds, labels)
# #             loss = nn.BCELoss(preds, labels)
#         loss.backward()
#         optimizer.step()
            
            
#         if (epoch + 1) % print_freq == 0:
#             print(preds)
#             print('epoch {} loss {}'.format(epoch+1, loss.item()))
    
#         loss = 0
#     return model  # return trained model
# #</GRADED>

In [128]:
def train_classification_model(x, y, model, num_epochs, lr=1e-1, print_freq=100, batch_size=10):
    """Train loop for a neural network model. Please use the SGD optimizer, optim.SGD.
    
    Input:
        train_loader:    Data loader for the train set. 
                         Enumerate through to train with each batch.
        model:           nn.Model to be trained
        num_epochs:      number of epochs to train the model for
        lr:              learning rate for the optimizer
        print_freq:      frequency to display the loss
    
    Output:
        model:   nn.Module trained model
    """
    n, d = x.shape
    optimizer = optim.SGD(model.parameters(), lr=lr)  # create an SGD optimizer for the model parameters
    for epoch in range(num_epochs):
        batch_ids = np.random.choice(n, batch_size, replace=False)
        x_batch = torch.tensor(x[batch_ids].astype(np.float32))
        y_batch = torch.tensor(y[batch_ids].astype(np.float32)).long()
        
        optimizer.zero_grad()
        preds = model(x_batch)
        preds = softmax_probs(preds)
        loss = nn.functional.cross_entropy(preds, y_batch)
        loss.backward()
        optimizer.step()
            
        if (epoch + 1) % print_freq == 0:
#             print(preds)
            print('epoch {} loss {}'.format(epoch+1, loss.item()))
            
    return model  # return trained model

In [129]:
def mse_loss(y_pred, y_true):
    square_diff = torch.pow((y_pred-y_true), 2)
    mean_error = 0.5 * torch.mean(square_diff)
    return mean_error

In [130]:
# Test function
#<GRADED>
def test_classification_model(x,y, model):
    """Tests the accuracy of the model.
    
    Input:
        test_loader:      Data loader for the test set. 
                          Enumerate through to test each example.
        model:            nn.Module model being evaluate.
        
    Output:
        accuracy:         Accuracy of the model on the test set.
    """
    # Compute the model accuracy
    
    accuracy = 0
    
    numer = 0 
    demon = 0
    
    preds = model(torch.tensor(x.astype(np.float32)))
    pred_lf = softmax_probs(preds)
    results = torch.argmax(pred_lf, axis=1).numpy()
    print(results)
    print(y)
    numer = np.sum(results == y)
    denom = len(y)
    return numer / denom
    
    
    #for batch_idx, (imgs, labels) in enumerate(test_loader):
    for i in range(len(x)):
        
        vector = torch.tensor(x[i].astype(np.float32))
        label = torch.tensor(y[i].astype(np.float32)).long()
        
        pred = model(vector)
        
        #preds = np.round(preds.numpy())
        
        labels = labels.numpy()
        
        numer+= np.sum(preds.numpy() == labels)
        demon+= np.size(preds.numpy())

    accuracy = numer/demon
    
    
    return accuracy
#</GRADED>

In [131]:
test_file_name = leg + "_test.csv"

t_lf = pd.read_csv(test_file_name)

t_id_arr = np.array(t_lf['id'])

t_lf2 = clean_up_my_data_bro(t_lf, leg, data_type="test")

x_training col number  367
removed column  3  bc all data are the same
removed column  185  bc all data are the same
x_lf and y_lf now has 365 columns


<ipython-input-118-da01bc06e466>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['gait'][i] = 1
<ipython-input-118-da01bc06e466>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_lf['speed'][i] = 0


In [132]:

hidden_dims = [20, 10]
num_epochs = 5000
lr = 1e-2

x, y = x_lf2, y_lf2


# n, d = x.shape
# learn_boundary = int(n * 0.75)
# permutation = np.random.permutation(n)
# x_shuffled = x[permutation]
# y_shuffled = y[permutation]
# x_tr = x_shuffled[:learn_boundary]
# y_tr = y_shuffled[:learn_boundary]
# x_te = x_shuffled[learn_boundary:]
# y_te = y_shuffled[learn_boundary:]


mlp_model = MLPNet(input_dim=365, hidden_dims=hidden_dims, output_dim=2)
# mlp_model = train_classification_model(x_tr, y_tr, mlp_model, num_epochs=num_epochs, lr=lr)
mlp_model = train_classification_model(x_lf2, y_lf2, mlp_model, num_epochs=num_epochs, lr=lr,batch_size=20)

# print('the number of parameters', sum(parameter.view(-1).size()[0] for parameter in mlp_model.parameters()))



epoch 100 loss 0.7155368328094482
epoch 200 loss 0.7069039344787598
epoch 300 loss 0.6758319139480591
epoch 400 loss 0.667433500289917
epoch 500 loss 0.6432809829711914
epoch 600 loss 0.5934021472930908
epoch 700 loss 0.570562481880188
epoch 800 loss 0.5395019054412842
epoch 900 loss 0.4911775588989258
epoch 1000 loss 0.45344844460487366
epoch 1100 loss 0.4277164936065674
epoch 1200 loss 0.4145854413509369
epoch 1300 loss 0.37584030628204346
epoch 1400 loss 0.37347984313964844
epoch 1500 loss 0.36321157217025757
epoch 1600 loss 0.347774475812912
epoch 1700 loss 0.3531357944011688
epoch 1800 loss 0.3379807770252228
epoch 1900 loss 0.3343455195426941
epoch 2000 loss 0.3331158757209778
epoch 2100 loss 0.3316533863544464
epoch 2200 loss 0.33292239904403687
epoch 2300 loss 0.3252294361591339
epoch 2400 loss 0.32556718587875366
epoch 2500 loss 0.3286602795124054
epoch 2600 loss 0.32380276918411255
epoch 2700 loss 0.3208160996437073
epoch 2800 loss 0.32233649492263794
epoch 2900 loss 0.320313

In [133]:
# test_classification_model(x_te, y_te, mlp_model)

In [134]:
t_lf2= torch.tensor(t_lf2.astype(np.float32))

pred_lf = mlp_model(t_lf2)

In [135]:
# print(y_lf2)
# print(pred_lf)
pred_lf = softmax_probs(pred_lf)
# print(pred_lf)
results = torch.argmax(pred_lf, axis=1).numpy()
print(results)

[0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1
 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0]


In [136]:
final_lf = results

# STEP 2.2: Use sklearn

In [137]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier # Strength of the L2 regularization term

hidden_layer_sizes = (20,10)
alpha = 1e-5 
batch_size = 20
max_iter = 5000
solver = "adam" # solver{‘lbfgs’} will not use mini-batches

# random_state is a hyper parameter but prolly not important


mlp = MLPClassifier(solver=solver, alpha=alpha, activation = 'relu',
                     hidden_layer_sizes=hidden_layer_sizes, random_state=None,batch_size =batch_size, max_iter = max_iter)

model = mlp.fit(x_lf2, y_lf2)



In [138]:
sk_result = model.predict(t_lf2)
print(sk_result)

[0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1
 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1 1 0 0 0]


In [139]:
print("checking the differences between P8 code and sk_learn base estimator")
np.array(sk_result)-np.array(results)

checking the differences between P8 code and sk_learn base estimator


array([ 0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0, -1,  0,
        0,  0,  0,  0,  0,  0])

In [140]:
# Now use adaboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# more hyper parameters
learning_rate = 1
n_estimators = 50

# Doesn't work; see below
#ada_boost = AdaBoostClassifier(base_estimator=mlp, n_estimators=n_estimators, 
#                               random_state=None, learning_rate = learning_rate)



In [141]:
# THIS DOESN'T WORK
# https://stackoverflow.com/questions/55632010/using-scikit-learns-mlpclassifier-in-adaboostclassifier

# ada_boost.fit(x_lf2,y_lf2)

In [142]:
# stackoverflow + chat gpt; perhaps u can use gpt4 lol

class customMLPClassifer(MLPClassifier):
    
    def __init__(self, hidden_layer_sizes, alpha, batch_size, max_iter,solver,random_state):
        
        super().__init__(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, 
                         batch_size=batch_size, 
                         max_iter=max_iter, 
                         solver=solver, 
                         random_state=random_state)
        
        # hopefully this works
    
    
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))
        

In [143]:
# custom_mlp = customMLPClassifer()


base_estimator = customMLPClassifer(solver=solver, alpha=alpha, 
                     hidden_layer_sizes=hidden_layer_sizes, random_state=None,batch_size =batch_size, max_iter = max_iter)

adabooster = AdaBoostClassifier(base_estimator= base_estimator, 
                                n_estimators=n_estimators, 
                               random_state=None, learning_rate = learning_rate)

In [144]:
adabooster.fit(x_lf2,y_lf2)

<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
<ipython-input-142-b87a0d6a7f67>:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

AdaBoostClassifier(base_estimator=customMLPClassifer(alpha=1e-05, batch_size=20,
                                                     hidden_layer_sizes=(20,
                                                                         10),
                                                     max_iter=5000,
                                                     random_state=None,
                                                     solver='adam'),
                   learning_rate=1)

In [145]:
sk_result_boosted = adabooster.predict(t_lf2)
print(sk_result_boosted)

[0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1
 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0]


In [146]:
print("checking the differences between P8 code and sk_learn BOOSTED estimator")
print(np.array(results)-np.array(sk_result_boosted))
print("\n")
print("checking the differences between sk_learn base estimator and sk_learn BOOSTED estimator")
print(np.array(sk_result_boosted)-np.array(sk_result))

checking the differences between P8 code and sk_learn BOOSTED estimator
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


checking the differences between sk_learn base estimator and sk_learn BOOSTED estimator
[ 0  0  0 -1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  1  0  0  0  0  0
  0  0]


In [147]:
final_lf = sk_result_boosted

In [148]:
print(final_lf)

[0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1
 0 0 1 0 0 0 1 0 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 0]


# STEP 3: Make Output Files

In [149]:
import csv

rows = zip(t_id_arr, final_lf)


In [150]:
# Write the tuples to a CSV file

csv_file_name = leg + "_test_labels.csv"

with open(csv_file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['id', leg]) # write header row
    writer.writerows(rows)

In [153]:
def make_sub():

    """
    This is a file that will help you convert your individual predictions to the final prediction. 
    In the same directory as this file, you should have the following 4 files:
      - LF_test_labels.csv - with at least two columns, 'id' and 'LF'
      - LH_test_labels.csv - with at least two columns, 'id' and 'LH'
      - RF_test_labels.csv - with at least two columns, 'id' and 'RF'
      - RH_test_labels.csv - with at least two columns, 'id' and 'RH'

    Running this script will convert these four files into a single CSV file, submission.csv, by
    mutating the IDs so that they also include the leg that is being checked.
    """

    legs = ["LF", "LH", "RF", "RH"]

    dfs = []

    for leg in legs:
        # read in the file
        test_prediction = pd.read_csv(f"{leg}_test_labels.csv")
        # append the abbreviation for the leg
        test_prediction['id'] = test_prediction['id'].astype(str) + f"_{leg}"
        # rename the label column
        test_prediction['label'] = test_prediction[leg]
        # exclude any potential additional columns
        dfs.append(test_prediction[['id', 'label']])

    final_df = pd.concat(dfs)
    final_df.to_csv("submission.csv", index=False)



In [154]:
make_sub()